In [1]:
import os
import time

import jax
import jax.numpy as jnp
import jax.random as jr
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

from mcmc import run_lmc_numpyro


matplotlib.use("Agg")  # noqa: E402


# a two-layer bayesian neural network with computational flow
# given by D_X => D_H => D_H => D_Y where D_H is the number of
# hidden units. (note we indicate tensor dimensions in the comments)
def model(X, Y, D_H, D_Y=1):
    N, D_X = X.shape

    # sample first layer (we put unit normal priors on all weights)
    w1 = numpyro.sample("w1", dist.Normal(jnp.zeros((D_X, D_H)), jnp.ones((D_X, D_H))))
    assert w1.shape == (D_X, D_H)
    z1 = jnp.tanh(jnp.matmul(X, w1))  # <= first layer of activations
    assert z1.shape == (N, D_H)

    # sample second layer
    w2 = numpyro.sample("w2", dist.Normal(jnp.zeros((D_H, D_H)), jnp.ones((D_H, D_H))))
    assert w2.shape == (D_H, D_H)
    z2 = jnp.tanh(jnp.matmul(z1, w2))  # <= second layer of activations
    assert z2.shape == (N, D_H)

    # sample final layer of weights and neural network output
    w3 = numpyro.sample("w3", dist.Normal(jnp.zeros((D_H, D_Y)), jnp.ones((D_H, D_Y))))
    assert w3.shape == (D_H, D_Y)
    z3 = jnp.matmul(z2, w3)  # <= output of the neural network
    assert z3.shape == (N, D_Y)

    if Y is not None:
        assert z3.shape == Y.shape

    # we put a prior on the observation noise
    prec_obs = numpyro.sample("prec_obs", dist.Gamma(3.0, 1.0))
    sigma_obs = 1.0 / jnp.sqrt(prec_obs)

    # observe data
    with numpyro.plate("data", N):
        # note we use to_event(1) because each observation has shape (1,)
        numpyro.sample("Y", dist.Normal(z3, sigma_obs).to_event(1), obs=Y)


# helper function for HMC inference
def run_inference(model, num_warmup, num_samples, num_chains, rng_key, X, Y, D_H):
    start = time.time()
    kernel = NUTS(model)
    mcmc = MCMC(
        kernel,
        num_warmup=num_warmup,
        num_samples=num_samples,
        num_chains=num_chains,
        progress_bar=False if "NUMPYRO_SPHINXBUILD" in os.environ else True,
    )
    mcmc.run(rng_key, X, Y, D_H)
    mcmc.print_summary()
    print("\nMCMC elapsed time:", time.time() - start)
    return mcmc.get_samples()


def run_inference_lmc(model, num_warmup, num_samples, num_chains, rng_key, X, Y, D_H):
    start = time.time()
    samples, steps = run_lmc_numpyro(
        jr.key(0),
        model,
        (X, Y, D_H),
        num_particles=num_chains,
        chain_len=num_samples,
        warmup_mult=num_warmup,
        tol=0.1,
        chain_sep=0.25,
    )
    elapsed = time.time() - start
    print(f"\nLMC elapsed time: {elapsed:.4}, num_steps: {steps}")
    return samples


# helper function for prediction
def predict(model, rng_key, samples, X, D_H):
    model = handlers.substitute(handlers.seed(model, rng_key), samples)
    # note that Y will be sampled in the model because we pass Y=None here
    model_trace = handlers.trace(model).get_trace(X=X, Y=None, D_H=D_H)
    return model_trace["Y"]["value"]


# create artificial regression dataset
def get_data(N=50, D_X=3, sigma_obs=0.05, N_test=500):
    D_Y = 1  # create 1d outputs
    np.random.seed(0)
    X = jnp.linspace(-1, 1, N)
    X = jnp.power(X[:, np.newaxis], jnp.arange(D_X))
    W = 0.5 * np.random.randn(D_X)
    Y = jnp.dot(X, W) + 0.5 * jnp.power(0.5 + X[:, 1], 2.0) * jnp.sin(4.0 * X[:, 1])
    Y += sigma_obs * np.random.randn(N)
    Y = Y[:, np.newaxis]
    Y -= jnp.mean(Y)
    Y /= jnp.std(Y)

    assert X.shape == (N, D_X)
    assert Y.shape == (N, D_Y)

    X_test = jnp.linspace(-1.3, 1.3, N_test)
    X_test = jnp.power(X_test[:, np.newaxis], jnp.arange(D_X))

    return X, Y, X_test


def main(
    num_data=50,
    num_hidden=5,
    num_samples=2000,
    num_warmup=1000,
    num_chains=1,
    use_lmc=False,
):
    N, D_X, D_H = num_data, 3, num_hidden
    X, Y, X_test = get_data(N=N, D_X=D_X)

    # do inference
    rng_key, rng_key_predict = jr.split(jr.PRNGKey(0))

    inference_fun = run_inference_lmc if use_lmc else run_inference
    samples = inference_fun(
        model, num_warmup, num_samples, num_chains, rng_key, X, Y, D_H
    )

    # predict Y_test at inputs X_test
    vmap_args = (
        samples,
        jr.split(rng_key_predict, num_samples * num_chains),
    )
    predictions = jax.jit(
        jax.vmap(lambda samples, rng_key: predict(model, rng_key, samples, X_test, D_H))
    )(*vmap_args)
    predictions = predictions[..., 0]

    # compute mean prediction and confidence interval around median
    mean_prediction = jnp.mean(predictions, axis=0)
    percentiles = np.percentile(predictions, [5.0, 95.0], axis=0)

    # make plots
    fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

    # plot training data
    ax.plot(X[:, 1], Y[:, 0], "kx")
    # plot 90% confidence level of predictions
    ax.fill_between(
        X_test[:, 1], percentiles[0, :], percentiles[1, :], color="lightblue"
    )
    # plot mean prediction
    ax.plot(X_test[:, 1], mean_prediction, "blue", ls="solid", lw=2.0)
    ax.set(xlabel="X", ylabel="Y", title="Mean predictions with 90% CI")

    plt.savefig("bnn_plot.pdf")

In [2]:
N, D_X, D_H = 50, 3, 5
X, Y, X_test = get_data(N=N, D_X=D_X)

# do inference
rng_key, rng_key_predict = jr.split(jr.PRNGKey(0))

samples = run_inference_lmc(model, 16, 2**8, 2**5, rng_key, X, Y, D_H)

100.00%|██████████| [00:01<00:00, 82.77%/s]
100.00%|██████████| [01:00<00:00,  1.65%/s]

LMC: gradient evaluations per output: 81.54

LMC elapsed time: 91.84, num_steps: 81.538330078125


In [3]:
import jax.tree_util as jtu


print(jtu.tree_map(lambda x: x.shape, samples))

{'prec_obs': (32, 256), 'w1': (32, 256, 3, 5), 'w2': (32, 256, 5, 5), 'w3': (32, 256, 5, 1)}


In [2]:
main(use_lmc=True)

100.00%|██████████| [11:05<00:00,  6.65s/%]
100.00%|██████████| [32:22<00:00, 19.43s/%]


LMC: gradient evaluations per output: 367.9

LMC elapsed time: 2.635e+03, num_steps: 367.8780212402344


ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (4 of them) had size 1, e.g. axis 0 of argument samples['prec_obs'] of type float32[1,2000];
  * one axis had size 2000: axis 0 of argument rng_key of type uint32[2000,2]

In [4]:
main()

sample: 100%|██████████| 3000/3000 [07:50<00:00,  6.37it/s, 511 steps of size 7.39e-03. acc. prob=0.85] 



                mean       std    median      5.0%     95.0%     n_eff     r_hat
  prec_obs     11.56      2.41     11.39      7.50     15.27   2071.61      1.00
   w1[0,0]      0.10      1.16      0.18     -1.72      2.07    294.63      1.00
   w1[0,1]     -0.07      1.13     -0.15     -1.89      1.83    265.78      1.02
   w1[0,2]     -0.10      1.16     -0.14     -1.89      1.78    343.29      1.01
   w1[0,3]     -0.09      1.21     -0.18     -1.93      1.98    302.05      1.00
   w1[0,4]      0.08      1.11      0.09     -1.61      1.99    255.54      1.00
   w1[1,0]     -0.10      1.16     -0.15     -1.83      1.75    177.52      1.00
   w1[1,1]      0.14      1.12      0.21     -1.63      1.83    106.35      1.06
   w1[1,2]      0.01      1.13      0.03     -1.71      1.89    323.59      1.00
   w1[1,3]      0.19      1.13      0.30     -1.69      1.87    286.64      1.01
   w1[1,4]     -0.07      1.17     -0.12     -1.83      1.82    200.40      1.00
   w1[2,0]     -0.13      1